In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup

import inspect
import re
import time

driver = webdriver.Chrome()
driver.get("https://x.com/")


In [2]:
def go_to_following_page():
    driver.get("https://x.com/")
    time.sleep(10)
    profile = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Profile'][data-testid='AppTabBar_Profile_Link']")
    profile.click()
    time.sleep(5)
    followings = driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-1rtiivn > a[href='/Mehr_mihan/following']")
    followings.click()
    time.sleep(5)

In [3]:
def go_down(n):
    try:
        # Create ActionChains object
        actions = ActionChains(driver)

        # Scroll down by simulating "Page Down" key press
        for _ in range(n):  # Adjust the range for the number of scrolls needed
            driver.execute_script("window.scrollBy(0, window.innerHeight);")
            time.sleep(4)  # Adjust the pause as needed to allow content to load

        return True
    except Exception as e:
        print(e)
        return False


In [4]:
def following_list():
    try:
        return driver.find_elements(By.CSS_SELECTOR, "div[data-testid='cellInnerDiv']")
    except:
        return False

In [5]:
def no_follow_back_list(list_of_followings):
    try:
        # List to hold items that didn't give a follow back
        no_follow_back_list = []

        # Iterate through each list item
        for following in list_of_followings:
            try:
                # Try to find the "Follows you" indicator within the following
                following.find_element(By.CSS_SELECTOR, "div[data-testid='userFollowIndicator']")
            except:
                # If the indicator is not found, add the following to the list
                no_follow_back_list.append(following)
        return no_follow_back_list
    
    except:
        return False


In [6]:
def unfollow_the_list(list_of_followings):
    try:
        list_for_unfollowing = no_follow_back_list(list_of_followings)
        if list_for_unfollowing:
            for item in list_for_unfollowing:
                try:
                    unfollow_button = item.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-1cwvpvk > button[aria-label^='Following @'][role='button']")
                    driver.execute_script("arguments[0].scrollIntoView(true);", unfollow_button)
                    time.sleep(2)  # Optional: Add a small delay to ensure the element is in view
                    # Click the button using JavaScript
                    driver.execute_script("arguments[0].click();", unfollow_button)
                    time.sleep(2)
                    confirm_unfollow = driver.find_element(By.CSS_SELECTOR, "button.css-175oi2r[data-testid='confirmationSheetConfirm'][role='button']")
                    confirm_unfollow.click()
                    time.sleep(random.randint(2, 10))
                except:
                    continue
        return True
    except:
        return False

In [7]:
def is_the_true_page():
    try:
        driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-14tvyh0.r-cpa5s6.r-16y2uox > a[href='/Mehr_mihan/verified_followers'][role='tab']")
        driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-14tvyh0.r-cpa5s6.r-16y2uox > a[href='/Mehr_mihan/followers'][role='tab']")
        driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-14tvyh0.r-cpa5s6.r-16y2uox > a[href='/Mehr_mihan/following'][role='tab'][aria-selected='true']")
        return True
    except:
        return False

In [10]:
def main():
    try:
        if not is_the_true_page():
            go_to_following_page()
            go_down(50)
            
        go_down(1)            
        list_of_followings = following_list()
        if unfollow_the_list(list_of_followings):
            return True
        else:
            return False
        
    except:
        return False

In [11]:
if __name__ == '__main__':
    go_to_following_page()
    go_down(50)
    
    while True:
        if main():
            time.sleep(5)
        else:
            driver.refresh()
            go_to_following_page()
            go_down(50)

KeyboardInterrupt: 